<a href="https://colab.research.google.com/github/clionelove123/temp_test/blob/main/Chap_02_CNN_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [10]:
train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 학습 데이터
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 테스트 데이터
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=50, shuffle=True)

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 4 * 4 * 50) # [batch_size, 50, 4, 4]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
cnn = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

In [ ]:
cnn.train()  # 학습을 위함
for epoch in range(10):
  for index, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()  # 기울기 초기화
    output = cnn(data)
    loss = criterion(output, target)
    loss.backward()  # 역전파
    optimizer.step()

    if index % 100 == 0:
      print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))

loss of 0 epoch, 0 index : 2.309504508972168
loss of 0 epoch, 100 index : 1.1938707828521729
loss of 0 epoch, 200 index : 0.45126283168792725
loss of 0 epoch, 300 index : 0.4104810655117035
loss of 0 epoch, 400 index : 0.2208472490310669
loss of 0 epoch, 500 index : 0.27360519766807556
loss of 0 epoch, 600 index : 0.28976935148239136
loss of 0 epoch, 700 index : 0.17999757826328278
loss of 0 epoch, 800 index : 0.2731815278530121
loss of 0 epoch, 900 index : 0.09685253351926804
loss of 0 epoch, 1000 index : 0.2798304259777069
loss of 0 epoch, 1100 index : 0.11062651872634888
loss of 1 epoch, 0 index : 0.20188535749912262
loss of 1 epoch, 100 index : 0.14810486137866974
loss of 1 epoch, 200 index : 0.07559692114591599
loss of 1 epoch, 300 index : 0.15267030894756317
loss of 1 epoch, 400 index : 0.1904611587524414
loss of 1 epoch, 500 index : 0.08527517318725586
loss of 1 epoch, 600 index : 0.1888185739517212
loss of 1 epoch, 700 index : 0.1393299102783203
loss of 1 epoch, 800 index : 0.1

In [ ]:
cnn.eval()  # test case 학습 방지를 위함
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    output = cnn(data)
    test_loss += criterion(output, target).item() # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))